# Image Moderation with Human-in-the-loop

Amazon Augmented AI (Amazon A2I) makes it easy to build the workflows required for human review of ML predictions. Amazon A2I brings human review to all developers, removing the undifferentiated heavy lifting associated with building human review systems or managing large numbers of human reviewers. 

Amazon A2I provides built-in human review workflows for common machine learning use cases, such as content moderation and text extraction from documents, which allows predictions from Amazon Rekognition and Amazon Textract to be reviewed easily. You can also create your own workflows for ML models built on Amazon SageMaker or any other tools. Using Amazon A2I, you can allow human reviewers to step in when a model is unable to make a high confidence prediction or to audit its predictions on an on-going basis. Learn more [here](https://aws.amazon.com/augmented-ai/).

In this tutorial, we will show how you can use Amazon A2I directly within your API calls to Rekognition's Image Moderation API. 

For more in depth instructions, visit [here](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-getting-started.html).

![image-moderation-hitl-arc](../images/image-moderation-hitl-architecture.png)

- [Step 1: Setup Notebook](#step1)
- [Step 2: Moderate image using Rekognition image moderation API](#step2)
- [Step 3: Set up A2I for human review](#step3)
    - [Step 3.1: Create Work Team](#step31)
    - [Step 3.2: Create Human Task UI](#step32)
    - [Step 3.3: Creating the Flow Definition](#step33)
- [Step 4: Moderate image with human-in-the-loop](#step4)
- [Step 5: Check A2I generated result in S3](#step5)

# Step 1: Setup Notebook <a id="step1"></a>

First, let's get the latest installations of our dependencies. You can ignore the warnings/errors.

In [ ]:
# First, let's get the latest installations of our dependencies
%pip install -qU pip
%pip install boto3 -qU
%pip install IPython -qU

In [ ]:
import boto3
import sagemaker as sm
import os
import datetime
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
import uuid
import json

# variables
data_bucket = sm.Session().default_bucket()
region = boto3.session.Session().region_name

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sm.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3')
rekognition=boto3.client('rekognition', region_name=region)
sagemaker = boto3.client('sagemaker')
a2i_runtime_client = boto3.client('sagemaker-a2i-runtime', region_name=region)

For this lab, we will use a sample image stored in this repo: ../datasets/yoga_swimwear.jpg to test Rekognition Image moderation API.

The image contains a lady in a bikini and Rekognition Image Moderation will label it as "Suggestive." 

![image-moderation-hitl-arc](../datasets/yoga_swimwear.jpg)

Now, let's upload the image to the default S3 bucket for Rekognition to access.

In [ ]:
s3_key = 'content-moderation-im/image-moderation/yoga_swimwear.jpg'
s3.upload_file('../datasets/yoga_swimwear.jpg', data_bucket, s3_key)

# Step 2: Moderate image using Rekognition image moderation API  <a id="step2"></a>
Call Rekognition Content Moderation API to detect inappropriate information in the image.

In [ ]:
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
    Image={
       'S3Object': {
           'Bucket': data_bucket,
           'Name': s3_key,
       }
    }
)
detectModerationLabelsResponse

As we can see in the Rekognition response, the Image Moderation API labeled the image in 3 categories with confidence scores:
- Top level category: **Suggestive** with a confidence score > 98%
- Second level category: **Female Swimwear Or Underwear** with a confidence score > 98%
- Second level category: **Revealing Clothes** with a confidence score ~ 67% 

The first 2 labels got high confidence scores > 98%. But the 3rd one, under Revealing Clothes, which has a low confidence score, may need a human to review and make a decision.

# Step 3: Set up A2I for human review <a id="step3"></a>

#### Setup Bucket CORS

**Important**: The bucket you specify for `data_bucket` must have CORS enabled. You can enable CORS by adding a policy similar to the following to your Amazon S3 bucket. To learn how to add CORS to an S3 bucket, see [CORS Permission Requirement](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-permissions-security.html#a2i-cors-update) in the Amazon A2I documentation. 


```
[{
   "AllowedHeaders": [],
   "AllowedMethods": ["GET"],
   "AllowedOrigins": ["*"],
   "ExposeHeaders": []
}]
```

If you do not add a CORS configuration to the S3 buckets that contains your image input data, human review tasks for those input data objects will fail. 

Print out the default S3 bucket name using the below cell if you don't know which bucket is in use.

In [ ]:
print('The default SageMaker Studio S3 bucket name: ' + data_bucket)

Run the below cell to enable CORS to the default S3 bucket

In [ ]:
cors_configuration = {
    'CORSRules': [{
       "AllowedHeaders": [],
       "AllowedMethods": ["GET"],
       "AllowedOrigins": ["*"],
       "ExposeHeaders": []
    }]
}

s3.put_bucket_cors(Bucket=data_bucket,
                   CORSConfiguration=cors_configuration)

## Step 3.1: Create Work Team <a id="step31"></a>
 A workforce is the group of workers that you have selected to label your dataset. You can choose either the Amazon Mechanical Turk workforce, a vendor-managed workforce, or you can create your own private workforce for human reviews. Whichever workforce type you choose, Amazon Augmented AI takes care of sending tasks to workers. When you use a private workforce, you also create work teams, a group of workers from your workforce that are assigned to Amazon Augmented AI human review tasks. You can have multiple work teams and can assign one or more work teams to each job. 

This lab assumes that you already have the workforce team in the same region. If you don't have a workforce team in the current region, follow [Step 1](https://catalog.us-east-1.prod.workshops.aws/workshops/1ece9ffd-4c24-4e66-b42a-0c0e13b0f668/en-US/content-moderation/01-image-moderation/02-image-moderation-with-a2i#step-1:-create-a-private-team-in-aws-console-(you-can-skip-this-step-if-you-already-have-a-private-work-team-in-the-region)) and [Step 2](https://catalog.us-east-1.prod.workshops.aws/workshops/1ece9ffd-4c24-4e66-b42a-0c0e13b0f668/en-US/content-moderation/01-image-moderation/02-image-moderation-with-a2i#step-2:-activate-a2i-user-account) on the Workshop website to set it up in the AWS console.

[More documents](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-workforce-management.html ) about how to set up a workforce team for A2I.

In [ ]:
# get the existing workforce arn
work_team_arn = sagemaker.list_workteams()["Workteams"][0]["WorkteamArn"]
work_team_arn

## Step 3.2: Create Human Task UI <a id="step32"></a>
Create a human task UI resource, giving a UI template in liquid html. This template will be rendered to the human workers whenever human loop is required. We are providing a simple demo template that is compatible with AWS Rekogition's Image Moderation API input and response. Since we are integrating A2I with Rekognition, we can create the template in the Console using default templates provided by [A2I](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-instructions-overview.html). To make things easier, the below template string is copied from the defeault template provided by Amazon A2I (found in the SageMaker Console under Worker task templates).

In [ ]:
template = """
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>
{% capture s3_uri %}s3://{{ task.input.aiServiceRequest.image.s3Object.bucket }}/{{ task.input.aiServiceRequest.image.s3Object.name }}{% endcapture %}

<crowd-form>
  <crowd-rekognition-detect-moderation-labels categories="[
      {% for label in task.input.selectedAiServiceResponse.moderationLabels %}
        {
          name: &quot;{{ label.name }}&quot;,
          parentName: &quot;{{ label.parentName }}&quot;,
        },
      {% endfor %}
    ]" src="{{ s3_uri | grant_read_access }}" header="Review the image and choose all applicable categories.">
    <short-instructions header="Instructions"><p>Review the image and choose all applicable categories.</p><p>If no categories apply, choose None.</p><p><br></p><p><strong>Nudity</strong></p><p>Visuals depicting nude male or female person or persons</p><p><br></p><p><strong>Graphic Male Nudity</strong></p><p>Visuals depicting full frontal male nudity, often close ups</p><p><br></p><p><strong>Graphic Female Nudity</strong></p><p>Visuals depicting full frontal female nudity, often close ups</p><p><br></p><p><strong>Sexual Activity</strong></p><p>Visuals depicting various types of explicit sexual activities and pornography</p><p><br></p><p><strong>Illustrated Explicit Nudity</strong></p><p>Visuals depicting animated or drawn sexual activity, nudity or pornography</p><p><br></p><p><strong>Adult Toys</strong></p><p>Visuals depicting adult toys, often in a marketing context</p><p><br></p><p><strong>Female Swimwear or Underwear</strong></p><p>Visuals depicting female person wearing only swimwear or underwear</p><p><br></p><p><strong>Male Swimwear Or Underwear</strong></p><p>Visuals depicting male person wearing only swimwear or underwear</p><p><br></p><p><strong>Barechested Male</strong></p><p>Visuals depicting topless males</p><p><br></p><p><strong>Partial Nudity</strong></p><p>Visuals depicting covered up nudity, for example using hands or pose</p><p><br></p><p><strong>Sexual Situations</strong></p><p>Visuals depicting passionate kissing and embracing of a sexual nature</p><p><br></p><p><strong>Revealing Clothes</strong></p><p>Visuals depicting revealing clothes and poses, such as deep cut dresses</p><p><br></p><p><strong>Graphic Violence or Gore</strong></p><p>Visuals depicting prominent blood or bloody injuries</p><p><br></p><p><strong>Physical Violence</strong></p><p>Visuals depicting violent physical assault, such as kicking or punching</p><p><br></p><p><strong>Weapon Violence</strong></p><p>Visuals depicting violence using weapons like firearms or blades, such as shooting</p><p><br></p><p><strong>Weapons</strong></p><p>Visuals depicting weapons like firearms and blades</p><p><br></p><p><strong>Self Injury</strong></p><p>Visuals depicting self-inflicted cutting on the body, typically in distinctive patterns using sharp objects</p><p><br></p><p><strong>Emaciated Bodies</strong></p><p>Visuals depicting extremely malnourished human bodies</p><p><br></p><p><strong>Corpses</strong></p><p>Visuals depicting human dead bodies</p><p><br></p><p><strong>Hanging</strong></p><p>Visuals depicting death by hanging</p><p><strong>Air Crash</strong> Visuals depicting air crashes <strong>Explosions and Blasts</strong> Visuals depicting blasts and explosions <strong>Middle Finger</strong> Visuals depicting a person showing the middle finger as a rude gesture <strong>Drug Products</strong> Visuals depicting drug products like joints or marijuana <strong>Drug Use</strong> Visuals depicting drug use, for example, snorting drug powders <strong>Pills</strong> Visuals depicting pills of any kind <strong>Drug Paraphernalia</strong> Visuals depicting drug paraphernalia like bongs and vaporizers <strong>Tobacco Products</strong> Visuals depicting tobacco products like cigarettes and e-cigarette devices <strong>Smoking</strong> Visuals depicting a person or persons smoking <strong>Drinking</strong> Visuals depicting a person or persons drinking alcoholic beverages <strong>Alcoholic Beverages</strong> Visuals depicting bottles or containers of alcoholic beverages <strong>Gambling</strong> Visuals depicting gambling, such as slot machines or casinos <strong>Nazi Party</strong> Visuals depicting Nazi party symbols, such as the Nazi Swastika <strong>White Supremacy</strong> Visuals depicting white supremacy symbols, such as the Confederate flag <strong>Extremist</strong> Visuals depicting flags and emblems of extremist organizations</p></short-instructions>

    <full-instructions header="Instructions"></full-instructions>
  </crowd-rekognition-detect-moderation-labels>
</crowd-form>

"""

def create_task_ui(task_ui_name):
    '''
    Creates a Human Task UI resource.

    Returns:
    struct: HumanTaskUiArn
    '''
    response = sagemaker.create_human_task_ui(
        HumanTaskUiName=task_ui_name,
        UiTemplate={'Content': template})
    return response

In [ ]:
# Task UI name - this value is unique per account and region. You can also provide your own value here.
taskUIName = 'content-moderation-im-image-ui-template'

# Create task UI
humanTaskUiResponse = create_task_ui(taskUIName)
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
print(humanTaskUiArn)

## Step 3.3: Creating the Flow Definition <a id="step33"></a>
In this section, we're going to create a flow definition. Flow Definitions allow us to specify:
- The conditions under which your human loop will be called. 
- The workforce that your tasks will be sent to. 
- The instructions that your workforce will receive. This is called a worker task template. 
- The configuration of your worker tasks, including the number of workers that receive a task and time limits to complete tasks. Where your output data will be stored.

This demo is going to use the API, but you can optionally create this workflow definition in the console as well. For more details and instructions, see [this document](https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-create-flow-definition.html).

### Specify Human Loop Activation Conditions
Since we are using a built-in integration type for A2I, we can use Human Loop Activation Conditions to provide conditions that trigger a human loop.Here we are specifying conditions for specific labels in our image. If Rekognition's confidence falls outside of the thresholds set here, the imagee will be sent to a human for review, with the specific keys that triggered the human loop prompted to the worker. 

In [ ]:
def create_flow_definition(flow_definition_name, a2i_output_path):
    '''
    Creates a Flow Definition resource

    Returns:
    struct: FlowDefinitionArn
    '''
    # Visit https://docs.aws.amazon.com/sagemaker/latest/dg/a2i-human-fallback-conditions-json-schema.html for more information on this schema.
    humanLoopActivationConditions = json.dumps(
        {
          "Conditions": [
            {
              "And": [
                {
                  "ConditionType": "ModerationLabelConfidenceCheck",
                  "ConditionParameters": {
                    "ModerationLabelName": "*",
                    "ConfidenceLessThan": 100
                  }
                },
                {
                  "ConditionType": "ModerationLabelConfidenceCheck",
                  "ConditionParameters": {
                    "ModerationLabelName": "*",
                    "ConfidenceGreaterThan": 50
                  }
                }
              ]
            }
          ]
        }
    )

    response = sagemaker.create_flow_definition(
            FlowDefinitionName= flow_definition_name,
            RoleArn= role,
            HumanLoopConfig= {
                "WorkteamArn": work_team_arn,
                "HumanTaskUiArn": humanTaskUiArn,
                "TaskCount": 1,
                "TaskDescription": "Image moderation sample task description",
                "TaskTitle": "Image moderation sample task"
            },
            HumanLoopRequestSource={
                "AwsManagedHumanLoopRequestSource": "AWS/Rekognition/DetectModerationLabels/Image/V3"
            },
            HumanLoopActivationConfig={
                "HumanLoopActivationConditionsConfig": {
                    "HumanLoopActivationConditions": humanLoopActivationConditions
                }
            },
            OutputConfig={
                "S3OutputPath" : a2i_output_path
            }
        )
    
    return response['FlowDefinitionArn']

Now we are ready to create our Flow Definition!

In [ ]:
# Flow definition name - this value is unique per account and region. You can also provide your own value here.
uniqueId = str(uuid.uuid4())
flowDefinitionName = f'fd-rekognition-image-{uniqueId}' 
a2i_output_path = f's3://{data_bucket}/content-moderation-im/a2i-output' # Where A2I keeps the output data

flowDefinitionArn = create_flow_definition(flowDefinitionName, a2i_output_path)

In [ ]:
def describe_flow_definition(name):
    '''
    Describes Flow Definition

    Returns:
    struct: response from DescribeFlowDefinition API invocation
    '''
    return sagemaker.describe_flow_definition(
        FlowDefinitionName=name)

import time
# Describe flow definition - status should be active
for x in range(60):
    describeFlowDefinitionResponse = describe_flow_definition(flowDefinitionName)
    print(describeFlowDefinitionResponse['FlowDefinitionStatus'])
    if (describeFlowDefinitionResponse['FlowDefinitionStatus'] == 'Active'):
        print("Flow Definition is active")
        break
    time.sleep(2)

# Step 4: Moderate image with human review <a id="step4"></a>

Let's call the Rekognition Image Moderation API with Human Loop Configurations. The condition we set up earlier was to trigger A2I when any label has a confidence score greater than 50%. So it is guaranteed that we can review the image in the A2I console.

In [ ]:
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
    Image={
       'S3Object': {
           'Bucket': data_bucket,
           'Name': s3_key,
       }
    },
    HumanLoopConfig={
       "FlowDefinitionArn":flowDefinitionArn,
       "HumanLoopName": f"rek-default-loop-{str(uuid.uuid4())}",
       "DataAttributes":{"ContentClassifiers":["FreeOfPersonallyIdentifiableInformation"]}
    }
)

Check Status of Human Loop

In [ ]:
arr = detectModerationLabelsResponse["HumanLoopActivationOutput"]["HumanLoopArn"].split('/')
human_loop_name=arr[len(arr)-1]

describeHumanLoopResponse = a2i_runtime_client.describe_human_loop(HumanLoopName=human_loop_name)
print(f'\nHuman Loop Name: {human_loop_name}')
print(f'Human Loop Status: {describeHumanLoopResponse["HumanLoopStatus"]}')

A human loop task is now sent to A2I and ready for us to review. You can find the A2I console URL from SageMaker console under Augment AI -> Human review workforces under the "private" tab. 

Or use the below code to print out the A2I console URL:

In [ ]:
workteamName = work_team_arn[work_team_arn.rfind('/') + 1:]
print("Navigate to the private worker portal and do the tasks. Make sure you've invited yourself to your workteam!")
print('https://' + sagemaker.describe_workteam(WorkteamName=workteamName)['Workteam']['SubDomain'])

Login to the A2I console using the username/password you received earlier when setting up the Workforce team. You should see the task "Image moderation sample task" in the list. Choose the item and click on the "start working" button on the top right to start viewing the moderation result. You will see a review page like the below screenshot:

![a2i image moderation](../images/image-moderation-hitl-a2i.png)

As a reviewer, you can confirm the moderation categories, then click on the "Submit" button. A2I will store the reviewer input in the S3 output folder.

In [ ]:
a2i_resp = a2i_runtime_client.describe_human_loop(HumanLoopName=human_loop_name)
print("Human Loop task status: ", a2i_resp["HumanLoopStatus"])
print("Human Loop output: ", a2i_resp["HumanLoopOutput"]["OutputS3Uri"])

# Step 5 Check A2I generated JSON <a id="step5"></a>
Now, let's download the A2I output file and print it out:

In [ ]:
s3.download_file(data_bucket, a2i_resp["HumanLoopOutput"]["OutputS3Uri"].replace(f's3://{data_bucket}/',''), 'a2i-output-image-moderation.json')

The `humanAnswers` field in the JSON file contains the reviewer's input. The `inputContent` field contains the original data sent to A2I.

In [ ]:
with open('a2i-output-image-moderation.json','r') as f:
    print(json.dumps(json.loads(f.read()), indent=2))

# Conclusion
In this lab, we moderated an image using Rekognition Image Moderation API. And set up the A2I components to review the moderation result.